In [1]:
# import dependencies
#Load and inspect images using the Python Imaging Library (PIL)
from PIL import Image
import os
import numpy as np
from PIL import Image
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import requests
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras import regularizers

In [2]:
# importing training data set
path = 'augmented_xray_images.pkl'
with open(path, 'rb') as file:
    X_train = pickle.load(file)

In [3]:
# read y training labels for images from csv file
y_train = np.loadtxt('y_augmented_labels.txt')
# display y_train
y_train

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]])

In [4]:
# importing training data set
path = 'testing_xray_images.pkl'
with open(path, 'rb') as file:
    X_test = pickle.load(file)

In [5]:
# read y training labels for images from csv file
y_test = np.loadtxt('y_testing_labels.txt')
# display y_train
y_test

array([[0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [6]:
# define x shape
X_shape = X_train[0].shape
# print shape
X_shape

(224, 224, 1)

In [7]:
# Convert values to numpy arrays
X_train = np.array(X_train)
X_test= np.array(X_test)
y_train= np.array(y_train)
y_test= np.array(y_test)

In [8]:
# Reshape test data for the model
X_test_np = []
for img in X_test:
    # Add a channel dimension for grayscale images
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    # Append the image to the list
    X_test_np.append(img)

# Convert to numpy array
X_test_np = np.array(X_test_np)

In [ ]:
# define input shape
input_shape = X_test_np[0].shape
# Define CNN model
model = keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(32, (3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='sigmoid',kernel_regularizer=regularizers.l1(0.01)),    
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=regularizers.l1(0.01)),
    layers.Flatten(),
    layers.Dense(64, activation='relu',kernel_regularizer=regularizers.l1_l2(0.01)),
    layers.Dense(6, activation='softmax',)  # 6 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 15
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

Epoch 1/15
476/476 ━━━━━━━━━━━━━━━━━━━━ 163s 340ms/step - accuracy: 0.2155 - loss: 44.4559 - val_accuracy: 0.2628 - val_loss: 16.1840
Epoch 2/15
476/476 ━━━━━━━━━━━━━━━━━━━━ 161s 338ms/step - accuracy: 0.2280 - loss: 15.8965 - val_accuracy: 0.2628 - val_loss: 15.4507
Epoch 3/15
476/476 ━━━━━━━━━━━━━━━━━━━━ 159s 334ms/step - accuracy: 0.2305 - loss: 15.5226 - val_accuracy: 0.2628 - val_loss: 15.5810
Epoch 4/15
476/476 ━━━━━━━━━━━━━━━━━━━━ 159s 334ms/step - accuracy: 0.2305 - loss: 15.5385 - val_accuracy: 0.2628 - val_loss: 15.4614
Epoch 5/15
476/476 ━━━━━━━━━━━━━━━━━━━━ 159s 334ms/step - accuracy: 0.2312 - loss: 15.5486 - val_accuracy: 0.2628 - val_loss: 16.1233
Epoch 6/15
476/476 ━━━━━━━━━━━━━━━━━━━━ 158s 333ms/step - accuracy: 0.2322 - loss: 15.5563 - val_accuracy: 0.2628 - val_loss: 15.6654
Epoch 7/15
476/476 ━━━━━━━━━━━━━━━━━━━━ 161s 337ms/step - accuracy: 0.2251 - loss: 15.5596 - val_accuracy: 0.2628 - val_loss: 15.2835
Epoch 8/15
476/476 ━━━━━━━━━━━━━━━━━━━━ 163s 342ms/step - accu

In [ ]:
# define input shape
input_shape = X_test_np[0].shape
# Define CNN model
model1 = keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(32, (3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=regularizers.l1(0.01)),    
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=regularizers.l1(0.01)),
    layers.Flatten(),
    layers.Dense(64, activation='relu',kernel_regularizer=regularizers.l1_l2(0.01)),
    layers.Dense(128, activation='sigmoid',kernel_regularizer=regularizers.l1_l2(0.01)),
    layers.Dense(6, activation='softmax',)  # 6 classes
])

# Compile the model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 15
history = model1.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

In [ ]:
# define input shape
input_shape = X_test_np[0].shape
# Define CNN model
model = keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(64, (7, 7), strides=(2, 2), activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2),kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(192, (3, 3), activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2),kernel_regularizer=regularizers.l1(0.01)), 
    layers.Conv2D(128, (1, 1),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(256, (3, 3),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(256, (1, 1),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(512, (3, 3),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2),kernel_regularizer=regularizers.l1(0.01)), 
    layers.Conv2D(256, (1, 1),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(512, (3, 3),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(512, (1, 1),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(1024, (3, 3),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2),kernel_regularizer=regularizers.l1(0.01)), 
    layers.Conv2D(512, (1, 1),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(1024, (3, 3),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),        layers.Conv2D(512, (1, 1),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(1024, (3, 3),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(1024, (3, 3),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(1024, (3, 3), strides=(2, 2),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(1024, (3, 3),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),        layers.Conv2D(512, (1, 1),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Conv2D(1024, (3, 3),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),        layers.Conv2D(512, (1, 1),activation='relu', padding='same'kernel_regularizer=regularizers.l1(0.01)),    
    layers.Flatten(),
    layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l1_l2(0.01)),
    layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l1_l2(0.01)),
    layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l1_l2(0.01)),
    layers.Dense(6, activation='softmax',)  # 6 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 15
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

In [ ]:
# plotting the traing and testing loss vs epoch
import matplotlib.pyplot as plt
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
# Get training and test loss histories
training_accuracy = history.history['accuracy']
test_accuracy = history.history['val_accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)
# Visualize accuracy history
plt.plot(epoch_count, training_accuracy, 'r--')
plt.plot(epoch_count, test_accuracy, 'b-')
plt.legend(['Training accuracy', 'Test accuracy'])
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.show();
plt.show()

In [ ]:
import torch
import torchvision.models as models

# Example: Load a pretrained model (e.g., ResNet) for image classification
model_pretrain = models.resnet18(pretrained=True)

# Define the file path to save the model
save_path = 'pretrained_resnet18.pth'

# Save the model's state dictionary to a file
torch.save(model_pretrain.state_dict(), save_path)

In [ ]:
# Load the input image
input_image = X_test[3]
#Image.open('..\Data\X_Ray Image DataSet\Guns\B0009_0001.png').convert('RGB')  # Convert to RGB

# Preprocess the image back into jpg
# Convert the image data back to the range [0, 255]
input_image = Image.fromarray((input_image * 255).astype('uint8'))  # Convert to PIL Image

# Convert the image to grayscale
input_image_gray = input_image.convert('L')

# Display the input image
display(input_image_gray)

# Preprocess the image
preprocess = transforms.Compose([
    # Resize the image to 224x224 pixels
    transforms.Resize((224, 224)),  
    # Convert to grayscale with 3 channels
    transforms.Grayscale(num_output_channels=3),  
    # Convert PIL image to PyTorch tensor
    transforms.ToTensor(),           
    # Normalize the image
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_tensor = preprocess(input_image_gray)

# Create a PyTorch tensor
input_batch = input_tensor.unsqueeze(0)  

#  Load a pre-trained model for image classification
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    output = model(input_batch)

# output results
_, predicted_idx = torch.max(output, 1)
predicted_class = predicted_idx.item()

print("Predicted class:", predicted_class)